In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/forest-cover-type-prediction/train.csv
/kaggle/input/forest-cover-type-prediction/sampleSubmission.csv
/kaggle/input/forest-cover-type-prediction/test.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
%matplotlib inline

# import h2o
# from h2o.automl import H2OAutoML

# h2o.init()

# Read Dataset

In [3]:
from sklearn.model_selection import train_test_split

data_train = pd.read_csv('/kaggle/input/forest-cover-type-prediction/train.csv', index_col='Id')
data_test = pd.read_csv('/kaggle/input/forest-cover-type-prediction/test.csv', index_col='Id')

In [4]:


y = 'Cover_Type'

X_train = data_train.drop(columns=[y])
y_train = data_train[y]

In [5]:
print(X_train.shape)
print(y_train.shape)

(15120, 54)
(15120,)


In [6]:
uni = y_train.unique()
uni.sort()
print(uni)

[1 2 3 4 5 6 7]


# Start Training

In [7]:
"""
obtain the distribution of the test data by submitting prediction data 
with all the same purpose variables.
"""
type_ratio = np.array([0.37053, 0.49681, 0.05936, 0.00103, 0.01295, 0.02687, 0.03242])
class_weight = {i+1: type_ratio[i] for i in range(0, 7)}
print(class_weight)

{1: 0.37053, 2: 0.49681, 3: 0.05936, 4: 0.00103, 5: 0.01295, 6: 0.02687, 7: 0.03242}


In [8]:
%%time

rf = RandomForestClassifier(class_weight=class_weight,
                            n_estimators=100,
                           random_state=42)

rf.fit(X_train, y_train)

CPU times: user 2.93 s, sys: 12 ms, total: 2.94 s
Wall time: 2.94 s


RandomForestClassifier(bootstrap=True,
                       class_weight={1: 0.37053, 2: 0.49681, 3: 0.05936,
                                     4: 0.00103, 5: 0.01295, 6: 0.02687,
                                     7: 0.03242},
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=None, oob_score=False,
                       random_state=42, verbose=0, warm_start=False)

# Prediction

In [9]:
%%time
prediction_test = rf.predict(data_test)

submission = pd.DataFrame.from_dict({'ID': data_test.index.tolist(),
                                    'Cover_Type': prediction_test.tolist(),
                                    })

submission.to_csv('./submission.csv', index=False)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

CPU times: user 13.9 s, sys: 224 ms, total: 14.1 s
Wall time: 14.1 s


In [10]:
print(submission.head())

print()
!head submission.csv

      ID  Cover_Type
0  15121           1
1  15122           1
2  15123           2
3  15124           2
4  15125           2

ID,Cover_Type
15121,1
15122,1
15123,2
15124,2
15125,2
15126,2
15127,2
15128,1
15129,1


# Calculate Accuracy of Test dataset

In [11]:
from sklearn.datasets import fetch_covtype

dataset  = fetch_covtype()

In [12]:
print(len(dataset.target))
print(max(data_train.index))
print(min(data_train.index))

print(max(data_test.index))
print(min(data_test.index))

581012
15120
1
581012
15121


In [13]:
index_test = data_test.index - 1
y_test = dataset.target[index_test.tolist()]

print(accuracy_score(prediction_test, y_test))

0.7295349642688004
